In [1]:
# ingest_outlook_min.py
import sqlite3, hashlib, re
from bs4 import BeautifulSoup
import win32com.client as win32



In [2]:

DB = "mail.db"
SCHEMA = """
CREATE TABLE IF NOT EXISTS emails(
  id TEXT PRIMARY KEY, subject TEXT, from_addr TEXT,
  sent_at_utc TEXT, folder TEXT, body_text TEXT, body_html TEXT
);
"""
conn = sqlite3.connect(DB); conn.executescript(SCHEMA)

PR_INTERNET_MESSAGE_ID = "http://schemas.microsoft.com/mapi/proptag/0x1035001E"
outlook = win32.Dispatch("Outlook.Application").GetNamespace("MAPI")

def ensure_id(item):
    try:
        mid = item.PropertyAccessor.GetProperty(PR_INTERNET_MESSAGE_ID) or ""
    except Exception:
        mid = ""
    if mid: return mid
    raw = f"{item.SentOn}-{item.Subject}-{item.SenderEmailAddress}"
    return hashlib.sha1(raw.encode("utf-8","ignore")).hexdigest()

def bodies(item):
    html = (getattr(item,"HTMLBody","") or "")
    text = (getattr(item,"Body","") or "")
    if not text and html:
        text = BeautifulSoup(html, "lxml").get_text("\n")
    return text, html

def walk(folder):
    items = getattr(folder,"Items",None)
    if items:
        for i in range(1, items.Count+1):
            it = items.Item(i)
            if getattr(it,"Class",None) != 43:  # olMail
                continue
            eid = ensure_id(it)
            text, html = bodies(it)
            conn.execute("INSERT OR IGNORE INTO emails VALUES (?,?,?,?,?, ?,?)",
                         (eid, it.Subject or "", it.SenderEmailAddress or "", str(getattr(it,"SentOn", "")),
                          folder.FolderPath, text, html))
        conn.commit()
    subs = getattr(folder,"Folders",None)
    if subs:
        for j in range(1, subs.Count+1):
            walk(subs.Item(j))

for s in range(1, outlook.Stores.Count+1):
    walk(outlook.Stores.Item(s).GetRootFolder())

print("OK, rows:", conn.execute("SELECT COUNT(*) FROM emails").fetchone()[0])
conn.close()

com_error: (-2146959355, 'Ошибка при выполнении приложения-сервера', None, None)

In [3]:
import win32com.client as win32
app = win32.gencache.EnsureDispatch("Outlook.Application")  # генерирует обёртки
ns = app.GetNamespace("MAPI")
print("Stores:", ns.Stores.Count)  # > 0 — отлично
root = ns.Stores.Item(1).GetRootFolder()
print("Root:", root.FolderPath)


com_error: (-2146959355, 'Ошибка при выполнении приложения-сервера', None, None)